In [1]:
from __future__ import print_function, division
import torch
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import os
import argparse
from time import time
#from load_data import *
from torch.optim import lr_scheduler
#from tool_metrics import bboxes_iou

In [2]:
    class AdaptiveBatchNorm2d(nn.Module):
        def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True):
            super(AdaptiveBatchNorm2d, self).__init__()
            self.bn = nn.BatchNorm2d(num_features, eps, momentum, affine)
            self.a = nn.Parameter(torch.FloatTensor(1, 1, 1, 1))
            self.b = nn.Parameter(torch.FloatTensor(1, 1, 1, 1))

        def forward(self, x):
            return self.a * x + self.b * self.bn(x)

    class Mish(torch.nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x):
            x = x * (torch.tanh(torch.nn.functional.softplus(x)))
            return x

    class wR2(nn.Module):
        def __init__(self, num_classes=1000):
            super(wR2, self).__init__()
            hidden1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, padding=2, stride=2),
                AdaptiveBatchNorm2d(48),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden2 = nn.Sequential(
                nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(64),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(128),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=160, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(num_features=160),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden5 = nn.Sequential(
                nn.Conv2d(in_channels=160, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden6 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden7 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden8 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden9 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden10 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )

            self.features = nn.Sequential(
                hidden1,
                hidden2,
                hidden3,
                hidden4,
                hidden5,
                hidden6,
                hidden7,
                hidden8,
                hidden9,
                hidden10
            )
            self.classifier = nn.Sequential(
                nn.Linear(23232, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, num_classes),
            )

        def forward(self, x):
            x1 = self.features(x)
            x11 = x1.view(x1.size(0), -1)
            x = self.classifier(x11)
            return x

In [4]:
model = wR2(4)
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.load_state_dict(torch.load(r'C:\Users\marco\wR2_MishAdaBN\wR2_MishAdaBN.pth8'))
#model_conv = model_conv.cuda()

<All keys matched successfully>

In [33]:
print('Training these layers')
for name, param in model.named_parameters():
    #if param.requires_grad is True:
    if 'a' != name.split('.')[-1] and 'b' != name.split('.')[-1] and 'bn' != name.split('.')[-2]:
        param.requires_grad = False
    print(name, param.requires_grad)

Training these layers
module.features.0.0.weight False
module.features.0.0.bias False
module.features.0.1.a True
module.features.0.1.b True
module.features.0.1.bn.weight True
module.features.0.1.bn.bias True
module.features.1.0.weight False
module.features.1.0.bias False
module.features.1.1.a True
module.features.1.1.b True
module.features.1.1.bn.weight True
module.features.1.1.bn.bias True
module.features.2.0.weight False
module.features.2.0.bias False
module.features.2.1.a True
module.features.2.1.b True
module.features.2.1.bn.weight True
module.features.2.1.bn.bias True
module.features.3.0.weight False
module.features.3.0.bias False
module.features.3.1.a True
module.features.3.1.b True
module.features.3.1.bn.weight True
module.features.3.1.bn.bias True
module.features.4.0.weight False
module.features.4.0.bias False
module.features.4.1.a True
module.features.4.1.b True
module.features.4.1.bn.weight True
module.features.4.1.bn.bias True
module.features.5.0.weight False
module.features

In [6]:
child_counter = 0
for child in model.children():
    print("child", child_counter, "is:")
    print(child)
    child_counter += 1

child 0 is:
wR2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Mish()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Mish()
      (3): MaxPool2d(kernel_size=2, stride=1, padding=1, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affin

In [17]:
for child in model.children():
    for c in child.children():
        for d in c.children():
            for e in d.children():
                for param in e.parameters():
                    print(e, param.size())
        #break
    #break

Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) torch.Size([48, 3, 5, 5])
Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) torch.Size([48])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([1, 1, 1, 1])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([1, 1, 1, 1])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([48])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([48])
Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)) torch.Size([64, 48, 5, 5])
Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)) torch.Size([64])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([